In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
from datetime import datetime
import pandas as pd

petitions = pd.read_csv("WTP_dataset_1K.csv", header=0)

#For US petitions, we use title and content of text
petitions['fulltext'] = petitions['title'].astype('str')+' '+petitions['content'].astype('str')
tf_vectorizer = CountVectorizer(ngram_range=(1,1), stop_words='english', encoding='utf-8', decode_error='ignore')
tf_vectorizer.fit(petitions['fulltext'])

def getdate(s):
    #If the time is proper timestamp, and different format, as in the case of UK petitions, 
    # then use \\
    # datev = s.split("T")[0]
    # gdate = datetime.strptime(datev, '%Y-%m-%d')
    
    gdate = datetime.strptime(s, '%d-%b-%y')
    return gdate

def getsimilarity(y, datev):
    gdate = datetime.strptime(datev, '%d-%b-%y')
    redpet['opened_at_date'] = redpet['FCD'].apply(getdate)
    subset = redpet[redpet['opened_at_date']<gdate]
    tfidfs = tf_vectorizer.transform(subset['fulltext'])
    minv = 0
    for i, j in zip(tfidfs.toarray(), subset['opened_at_date']):        
        minv += (1 - cosine(i,y))/((float((gdate-j).days)*1.0)/7.0)
    return minv
        
    
    

In [ ]:
from random import randint
import matplotlib

import numpy as np
import torch, os

# We user infersent from https://github.com/facebookresearch/InferSent
# Set of rejected petitions title is given in resources/Noaction_title.csv
# We map the title of rejected petitions and petitions from evaluation set to a vector reoresentation using infersent

os.chdir('path/InferSent-master/')

GLOVE_PATH = 'glove.840B.300d.txt'
model = torch.load('encoder/infersent.allnli.pickle',map_location=lambda storage, loc: storage)
model.set_glove_path(GLOVE_PATH)

sentences = {}
x=0
with open('Petitionstitle') as f:
    for line in f:
        sentences[str(x)] = line.strip().decode("utf-8")
        x += 1

print(len(sentences))

model.build_vocab_k_words(K=100000)
model.update_vocab(sentences.values(), tokenize=True)

embeddings = model.encode(sentences.values(), bsize=1, tokenize=True, verbose=True)

print len(embeddings), len(embeddings[0][0])

writeout = []
for i in range(0, len(embeddings)):
        writeout.append(embeddings[i][0])

np.savetxt("Titleembeddings", writeout)


In [ ]:
from sklearn.svm import OneClassSVM
from sklearn import svm
import numpy as np
from sklearn.preprocessing import scale

x = np.loadtxt("NoactionTitle_embeddings")
pet = scale(np.loadtxt("Titleembeddings"))

clf = svm.OneClassSVM(kernel="linear", nu=0.01)
clf.fit(x)
petact = clf.decision_function(train)

#petition title action score
np.savetxt("petactscore", petact)
